# **1.라이브러리 import**

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import font_manager
from matplotlib import gridspec
import pprint
import re
import requests
from bs4 import BeautifulSoup
import time

import selenium
# 웹 드라이버를 초기화하고 웹 브라우저를 조작하는 데 필요한 클래스를 포함
from selenium import webdriver

# Chrome 웹 드라이버 서비스, 웹 브라우저의 옵션 설정
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options

# 웹 페이지에서 요소를 찾는 데 사용되는 다양한 기준(By.ID, By.NAME, By.XPATH)
from selenium.webdriver.common.by import By

# 웹 요소에 대한 마우스 및 키보드 조작
from selenium.webdriver.common.action_chains import ActionChains

# 특수 키보드 키(예: Enter, Tab)를 조작
from selenium.webdriver.common.keys import Keys

# 브라우저를 제어하기 위한 웹 드라이버 설정을 정의
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# 특정 조건이 충족될 때까지 대기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

# HTML <select> 요소(드롭다운 메뉴)를 조작
from selenium.webdriver.support.select import Select

# <웹 드라이버 관리를 위한 webdriver_manager 라이브러리를 설치>
# !pip install webdriver_manager
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager

# 경고 메세지 무시
import warnings
warnings.filterwarnings('ignore')

# 맷플롯립 그래프를 노트북 안에서 보여주도록 설정
%matplotlib inline

# **2.URL 파라미터 설정**

In [2]:
# key = '공공데이터 포털 개인 API 일반 Key 입력 <- 이걸 입력해야 코드 실행됨'

시작일 = '2020-10-01'
마지막일 = '2021-09-31'
지역들 = ['서울', '경기', '인천']

# **3. 공공데이터포털 API 데이터 스크래핑** 

In [3]:
# 데이터를 저장할 초기화된 딕셔너리 만들기
data_dict = {
    '타입': [],
    '아파트명': [],
    '지역': [],
    '도로명주소': [],
    '세대수': [],
    '시행사': [],
    '시공사': [],
    '세부정보': []    
}

# 지역들 = [서울, 경기, 인천] 데이터 수집
for 지역 in 지역들 :
    # API 요청 URL 생성
    url = f'https://api.odcloud.kr/api/ApplyhomeInfoDetailSvc/v1/getAPTLttotPblancDetail?page=1&perPage=200&cond[SUBSCRPT_AREA_CODE_NM::EQ]={지역}&cond[RCRIT_PBLANC_DE::LTE]={마지막일}&cond[RCRIT_PBLANC_DE::GTE]={시작일}&serviceKey={key}'
    # API 요청
    res = requests.get(url)
    # 응답 데이터 Json 파싱
    data = res.json()
    
    # 데이터 딕셔너리에 추가
    for i in range(data['currentCount']) :
        data_dict['타입'].append(data['data'][i]['HOUSE_SECD_NM'])
        data_dict['아파트명'].append(data['data'][i]['HOUSE_NM'])
        data_dict['지역'].append(data['data'][i]['SUBSCRPT_AREA_CODE_NM'])
        data_dict['도로명주소'].append(data['data'][i]['HSSPLY_ADRES'])
        data_dict['세대수'].append(data['data'][i]['TOT_SUPLY_HSHLDCO'])
        data_dict['시행사'].append(data['data'][i]['BSNS_MBY_NM'])
        data_dict['시공사'].append(data['data'][i]['CNSTRCT_ENTRPS_NM'])
        data_dict['세부정보'].append(data['data'][i]['PBLANC_URL'])

In [5]:
data_dict.keys()

dict_keys(['타입', '아파트명', '지역', '도로명주소', '세대수', '시행사', '시공사', '세부정보'])

In [4]:
# 수집한 청약 데이터 데이터프레임으로 변환한 후 확인 
apt = pd.DataFrame(data_dict)
apt.head(1)

,타입,아파트명,지역,도로명주소,세대수,시행사,시공사,세부정보
0,APT,신림스카이아파트,서울,"서울특별시 관악구 신림로 185(신림동, 신림 스카이 아파트)",43,신림 스카이 아파트,(주)제이더블유종합건설,https://www.applyhome.co.kr/ai/aia/selectAPTLt...


# **4. 부족한 데이터 셀레니움으로 가져오기**
- apt 세부정보 컬럼의 URL을 이용하여 크롤링하기

In [6]:
# 1. 데이터를 저장할 초기화된 딕셔너리
data_dict2 = {
    '타입': [],
    '아파트명': [],
    '지역': [],
    '도로명주소': [],
    '세대수': [],
    '시행사': [],
    '시공사': [],
    '주택형': [],
    '주택공급면적': [],
    '공급세대수': [],
    '공급금액(최고가 기준)': [],
    '입주예정월': []
}

# service = ChromeService(executable_path=ChromeDriverManager().install())

# 2. Chrome WebDriver를 설정
options = Options()

# 3. 크롬을 숨기는 옵션 추가
options.add_argument("--headless")
options.add_argument('--blink-settings=imagesEnabled=false')

# 4. 데이터를 많이 요청할 경우 User-Agent로 크롬 브라우저로 인식되도록 설정
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")


# 5. 크롬드라이버 실행
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)


# 6. apt의 '세부정보' 컬럼에서 URL을 가져와 순회
for num, url in enumerate(apt['세부정보']) :
    driver.get(url)
    table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.tbl_scroll')))

    # 표(table)의 tbody 요소 찾기
    tbody = table.find_element(By.TAG_NAME, 'tbody') 
    # tbody 하위의 모든 tr 요소 찾기
    rows = tbody.find_elements(By.TAG_NAME, 'tr') 

    # 두 번째 표(table2) 찾기
    table_xpath = "//*[@id='printArea']/table[2]"
    table2 = driver.find_element(By.XPATH, table_xpath)
    tbody2 = table2.find_element(By.TAG_NAME, 'tbody')  # tbody 요소 찾기
    rows2 = tbody2.find_elements(By.TAG_NAME, 'tr')  # tbody 하위의 모든 tr 요소 찾기

    # '세부정보'에서 날짜 정보(li_text) 추출
    li_xpath = "//*[@id='printArea']/ul[3]/li[1]"
    li_element = driver.find_element(By.XPATH, li_xpath)
    li_text = li_element.text[-7:]
    li_text = f'{li_text[:4]}-{li_text[5:]}-01'

    result_list = []
    result_list2 = []

    # 첫 번째 표에서 데이터를 추출
    for row in rows[:-1] :  # 마지막 줄 제외
        cells = row.find_elements(By.TAG_NAME, 'td')
        result_list.append([cell.text for cell in cells])
    
    # 두 번째 표에서 데이터를 추출
    for row2 in rows2:  # 마지막 줄 제외
        cells = row2.find_elements(By.TAG_NAME, 'td')
        result_list2.append([cell.text for cell in cells])    
        
    # 데이터를 딕셔너리에 추가
    for idx, val in enumerate(result_list) :
        if idx == 0 :
            val.pop(0)
        data_dict2['타입'].append(apt['타입'][num])
        data_dict2['아파트명'].append(apt['아파트명'][num])
        data_dict2['지역'].append(apt['지역'][num])
        data_dict2['도로명주소'].append(apt['도로명주소'][num])
        data_dict2['세대수'].append(apt['세대수'][num])
        data_dict2['시행사'].append(apt['시행사'][num])
        data_dict2['시공사'].append(apt['시공사'][num])
        data_dict2['주택형'].append(val[0])
        data_dict2['주택공급면적'].append(val[1])
        data_dict2['공급세대수'].append(val[4])

    # 두 번째 표의 데이터를 딕셔너리에 추가
    for val in result_list2 :
        data_dict2['공급금액(최고가 기준)'].append(val[1])

    # 날짜 정보를 딕셔너리에 추가
    for i in range(idx+1):
        data_dict2['입주예정월'].append(li_text)
        
    # 크롤링 간 시간 지연을 위해 3초 대기
    time.sleep(3)

In [22]:
apt2 = pd.DataFrame(data_dict2)
print('총 크롤링 데이터의 길이 : ', len(apt2))
apt2.head(5)

총 크롤링 데이터의 길이 :  1029


,타입,아파트명,지역,도로명주소,세대수,시행사,시공사,주택형,주택공급면적,공급세대수,공급금액(최고가 기준),입주예정월
0,APT,신림스카이아파트,서울,"서울특별시 관악구 신림로 185(신림동, 신림 스카이 아파트)",43,신림 스카이 아파트,(주)제이더블유종합건설,035.7100,42.8800,1,"45,000",2021-09-01
1,APT,신림스카이아파트,서울,"서울특별시 관악구 신림로 185(신림동, 신림 스카이 아파트)",43,신림 스카이 아파트,(주)제이더블유종합건설,038.3400,45.6900,9,"45,000",2021-09-01
2,APT,신림스카이아파트,서울,"서울특별시 관악구 신림로 185(신림동, 신림 스카이 아파트)",43,신림 스카이 아파트,(주)제이더블유종합건설,045.8500,54.2000,1,"53,000",2021-09-01
3,APT,신림스카이아파트,서울,"서울특별시 관악구 신림로 185(신림동, 신림 스카이 아파트)",43,신림 스카이 아파트,(주)제이더블유종합건설,049.5700,58.3100,1,"48,000",2021-09-01
4,APT,신림스카이아파트,서울,"서울특별시 관악구 신림로 185(신림동, 신림 스카이 아파트)",43,신림 스카이 아파트,(주)제이더블유종합건설,050.4600,59.4100,1,"57,000",2021-09-01


# **5. CSV로 저장**

In [9]:
# apt2.to_csv('2020년10월_2021년09월.csv', index=False, encoding='cp949')

# **6. excel로 저장**
- 크롤링 안된 데이터 구글 시트내에서 추가 데이터 수집하기 위함.

In [23]:
!pip install openpyxl

apt2.to_excel('2020년10월_2021년09월.xlsx')